# Linq

Inspired by engineer Spock


In [1]:
#r "nuget:AlgorithmLibrary.MicBai" 
using AlgorithmLibrary;

using System.IO;
using System.Text;
using System.Collections.Generic;
using System.Linq;

Error: C:\Users\micha\.packagemanagement\nuget\Projects\17716--c8bada07-4e0a-44b3-9f37-619418ec48ae\Project.fsproj : error NU1101: Das Paket "AlgorithmLibrary.MicBai" wurde nicht gefunden. Es sind keine Pakete mit dieser ID in folgenden Quellen vorhanden: C:\Program Files\dotnet\library-packs, C:\Program Files\dotnet\sdk\7.0.300-preview.23179.2\FSharp\library-packs, Microsoft Visual Studio Offline Packages, nuget.org, WaferTrim

In [3]:
public class ChessPlayer
{
    private string _country;
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public int BirthYear { get; set; }
    public int Rating { get; set; }
    public string Country
    {
        get
        {
            return _country;
        }
        set => _country = value;
    }
    public int Id { get; set; }
    
    public override string ToString()
    {
        return $"Full Name: {FirstName + " " + LastName}, Rating = {Rating}, from {Country}, Born in {BirthYear}";
    }

    public static ChessPlayer ParseFideCsv(string line)
    {
        string[] parts = line.Split(';');
        return new ChessPlayer()
        {
            Id = int.Parse(parts[0]),
            LastName = parts[1].Split(',')[0].Trim(),
            FirstName = parts[1].Split(',')[1].Trim(),
            Country = parts[3],
            Rating = int.Parse(parts[4]),
            BirthYear = int.Parse(parts[6])
        };
    }
}